# Meta-Agent Visualization Tutorial

Interactive tutorial for visualizing and analyzing hierarchical meta-agent systems.

This notebook demonstrates:
1. Setting up a meta-agent system
2. Running hierarchical evolution
3. Generating visualizations
4. Analyzing emergent dynamics
5. Exploring energy landscapes

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from meta.emergence import MultiScaleSystem, HierarchicalAgent
from meta.hierarchical_evolution import HierarchicalEvolutionEngine
from meta.participatory_diagnostics import ParticipatoryDiagnostics
from meta.visualization import (
    MetaAgentAnalyzer,
    HierarchyVisualizer,
    ConsensusVisualizer,
    DynamicsVisualizer
)
from meta.energy_visualization import EnergyVisualizer

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Imports successful!")

## 1. Create Multi-Scale System

We'll create a system with 8 agents at scale 0, initialized in two clusters.

In [ ]:
# Parameters
n_agents = 8
dim = 3  # Belief dimensionality
max_emergence_levels = 3

# Create initial agents
agents = []
for i in range(n_agents):
    # Create two clusters
    group = i // (n_agents // 2)
    mu_q = np.random.randn(dim) + group * 3.0  # Cluster centers at 0 and 3
    Sigma_q = np.eye(dim) * 0.5
    
    mu_p = np.zeros(dim)
    Sigma_p = np.eye(dim) * 5.0
    
    phi = np.eye(dim)  # Identity gauge frame
    
    agent = HierarchicalAgent(
        mu_q=mu_q,
        Sigma_q=Sigma_q,
        mu_p=mu_p,
        Sigma_p=Sigma_p,
        phi=phi,
        scale=0,
        local_index=i
    )
    agents.append(agent)

# Create system
system = MultiScaleSystem(
    agents={0: agents},
    max_emergence_levels=max_emergence_levels,
    max_meta_membership=n_agents,
    max_total_agents=n_agents * 10
)

print(f"✓ Created system with {n_agents} agents")
print(f"  Belief dimension: {dim}")
print(f"  Max emergence levels: {max_emergence_levels}")

## 2. Initialize Diagnostics and Analyzer

In [ ]:
# Create diagnostics tracker
diagnostics = ParticipatoryDiagnostics(
    system=system,
    track_energy=True,
    track_priors=True,
    track_gradients=False
)

# Create analyzer
analyzer = MetaAgentAnalyzer(system)

print("✓ Diagnostics and analyzer initialized")

## 3. Create Evolution Engine

In [ ]:
engine = HierarchicalEvolutionEngine(
    system=system,
    lr_mu_q=0.1,
    lr_Sigma_q=0.05,
    lr_mu_p=0.05,
    lr_Sigma_p=0.02,
    belief_kl_threshold=0.5,
    model_kl_threshold=0.5,
    timescale_filtering=True,
    deactivate_constituents=False,
    ouroboros_tower=False
)

print("✓ Evolution engine ready")
print(f"  Learning rates: μ_q={0.1}, Σ_q={0.05}")
print(f"  KL thresholds: {0.5}")

## 4. Run Evolution

We'll run 50 steps and capture snapshots every 5 steps.

In [ ]:
n_steps = 50
snapshot_interval = 5

print(f"Running evolution for {n_steps} steps...")

for step in range(n_steps):
    # Evolve
    engine.step()
    
    # Capture diagnostics every step
    diagnostics.capture_snapshot()
    
    # Capture analyzer snapshot at intervals
    if step % snapshot_interval == 0 or step == n_steps - 1:
        analyzer.capture_snapshot()
        
        # Progress update
        n_active = sum(
            len([a for a in agents if a.is_active])
            for agents in system.agents.values()
        )
        print(f"  Step {step:3d}: Scales={list(system.agents.keys())}, "
              f"Active={n_active}, Condensations={len(system.condensation_events)}")

print(f"\n✓ Evolution complete!")
print(f"  Total condensations: {len(system.condensation_events)}")
print(f"  Final scales: {sorted(system.agents.keys())}")
print(f"  Snapshots captured: {len(analyzer.snapshots)}")

## 5. Visualize Hierarchical Structure

In [ ]:
hierarchy_viz = HierarchyVisualizer(analyzer)

fig = hierarchy_viz.plot_hierarchy_tree(layout='hierarchical', figsize=(14, 10))
plt.show()

print("\nHierarchy visualization generated!")
print("- Nodes colored by scale (ζ)")
print("- Solid circles = active agents")
print("- Hollow circles = inactive agents")
print("- Edge thickness ∝ coherence")

## 6. Consensus Matrix Analysis

Visualize pairwise KL divergences to see which agents are in consensus.

In [ ]:
consensus_viz = ConsensusVisualizer(analyzer)

# Plot consensus matrix for scale 0
try:
    fig = consensus_viz.plot_consensus_matrix(scale=0, metric='belief')
    plt.show()
    
    print("\nConsensus matrix interpretation:")
    print("- Dark green: Low KL → agents in consensus")
    print("- Red: High KL → divergent beliefs")
    print("- Look for block patterns (clusters)")
except Exception as e:
    print(f"Could not plot consensus matrix: {e}")

## 7. Coherence Trajectories

Track how coherence evolves for each meta-agent.

In [ ]:
try:
    fig = consensus_viz.plot_coherence_trajectories(figsize=(14, 6))
    plt.show()
    
    print("\nCoherence trajectories:")
    print("- Each line = one meta-agent")
    print("- Red dashed line = high coherence threshold (0.95)")
    print("- Both high → 'epistemic death' → ready for condensation")
except ValueError as e:
    print(f"No meta-agents formed yet: {e}")

## 8. Multi-Scale Dynamics

### Scale Occupancy Heatmap

In [ ]:
dynamics_viz = DynamicsVisualizer(analyzer)

fig = dynamics_viz.plot_scale_occupancy(figsize=(14, 6))
plt.show()

print("\nScale occupancy heatmap:")
print("- X-axis: Time")
print("- Y-axis: Scale (ζ)")
print("- Color: Number of active agents")
print("- Stars (★): Condensation events")

### Population Flows

In [ ]:
fig = dynamics_viz.plot_population_flows(figsize=(12, 6))
plt.show()

print("\nPopulation flows:")
print("- Stacked area chart showing agent counts per scale")
print("- Watch for bottom-up flow (emergence)")

### Condensation Timeline

In [ ]:
if system.condensation_events:
    fig = dynamics_viz.plot_condensation_timeline(figsize=(14, 6))
    plt.show()
    
    print("\nCondensation timeline:")
    print("- Each point = one condensation event")
    print("- Size ∝ number of constituents")
    print("- Color = belief coherence")
else:
    print("No condensation events recorded.")

## 9. Energy Landscape Analysis

In [ ]:
energy_viz = EnergyVisualizer(diagnostics)

# Multi-scale energy landscape
fig = energy_viz.plot_energy_landscape(figsize=(14, 8))
plt.show()

print("\nEnergy landscape:")
print("- Separate subplot for each scale")
print("- Red: E_self (intrinsic energy)")
print("- Cyan: E_belief_align (lateral coherence)")
print("- Blue: E_prior_align (vertical alignment)")
print("- Black line: Total energy")

### Energy Flow (Flux)

In [ ]:
try:
    fig = energy_viz.plot_energy_flow(figsize=(12, 6))
    plt.show()
    
    print("\nEnergy flow:")
    print("- dE/dt for each scale")
    print("- Positive = energy increasing")
    print("- Negative = energy decreasing")
except ValueError as e:
    print(f"Not enough snapshots for energy flow: {e}")

### Prior Evolution (Top-Down Information Flow)

In [ ]:
try:
    fig = energy_viz.plot_prior_evolution(figsize=(14, 6))
    plt.show()
    
    print("\nPrior evolution:")
    print("- KL divergence between consecutive priors")
    print("- Measures top-down influence from meta-agents")
    print("- Log scale on Y-axis")
except ValueError as e:
    print(f"No prior changes recorded: {e}")

## 10. Detailed Event Analysis

Let's examine the condensation events in detail.

In [ ]:
if system.condensation_events:
    print("Condensation Events:")
    print("=" * 70)
    
    for i, event in enumerate(system.condensation_events):
        print(f"\nEvent {i+1}:")
        print(f"  Time:              {event['time']}")
        print(f"  Scales:            {event['source_scale']} → {event['target_scale']}")
        print(f"  Constituents:      {event['n_constituents']}")
        print(f"  Belief coherence:  {event['coherence']['belief']:.4f}")
        print(f"  Model coherence:   {event['coherence']['model']:.4f}")
        print(f"  Leader score:      {event['leader_score']:.4f}")
        print(f"  Leader index:      {event['leader_index']}")
else:
    print("No condensation events occurred.")
    print("Try:")
    print("  - Running more steps")
    print("  - Lowering KL thresholds")
    print("  - Increasing learning rates")

## 11. System Summary Statistics

In [ ]:
print("System Summary")
print("=" * 70)

final_snapshot = analyzer.snapshots[-1]

print(f"\nFinal State (t={final_snapshot.time}):")
print(f"  Total agents:     {final_snapshot.metrics['total_agents']}")
print(f"  Active agents:    {final_snapshot.metrics['total_active']}")
print(f"  Max scale:        {final_snapshot.metrics['max_scale']}")
print(f"  Condensations:    {len(system.condensation_events)}")

print(f"\nAgents by scale:")
for scale in sorted(final_snapshot.metrics['n_agents_by_scale'].keys()):
    n_total = final_snapshot.metrics['n_agents_by_scale'][scale]
    n_active = final_snapshot.metrics['n_active_by_scale'][scale]
    n_meta = final_snapshot.metrics['n_meta_by_scale'][scale]
    print(f"  ζ={scale}: {n_total} total, {n_active} active, {n_meta} meta")

print(f"\nEnergy Evolution:")
if diagnostics.energy_snapshots:
    first = diagnostics.energy_snapshots[0]
    last = diagnostics.energy_snapshots[-1]
    
    E_init = sum(s['E_total'] for s in first['by_scale'].values())
    E_final = sum(s['E_total'] for s in last['by_scale'].values())
    
    print(f"  Initial total:    {E_init:.2f}")
    print(f"  Final total:      {E_final:.2f}")
    print(f"  Change:           {E_final - E_init:.2f} ({100 * (E_final - E_init) / E_init:.1f}%)")
else:
    print("  No energy data")

## 12. Export Data

Save all data for future analysis.

In [ ]:
# Export snapshots to JSON
analyzer.export_to_json('./tutorial_snapshots.json')

print("✓ Data exported to './tutorial_snapshots.json'")
print("\nThis file contains:")
print("  - All system snapshots")
print("  - Agent states (beliefs, priors, etc.)")
print("  - Meta-agent descriptors")
print("  - Condensation events")
print("  - System configuration")

## 13. Interactive Visualizations (Optional)

If Plotly is installed, generate interactive HTML visualizations.

In [ ]:
try:
    # Interactive hierarchy
    interactive_fig = hierarchy_viz.plot_interactive_hierarchy()
    
    if interactive_fig:
        interactive_fig.write_html('./tutorial_hierarchy_interactive.html')
        print("✓ Interactive hierarchy saved to './tutorial_hierarchy_interactive.html'")
        print("  Open in browser for exploration")
        
        # Display in notebook (if in Jupyter)
        interactive_fig.show()
except Exception as e:
    print(f"Interactive visualization not available: {e}")
    print("Install Plotly: pip install plotly")

## Next Steps

Try experimenting with:

1. **Different initial conditions**
   - More/fewer agents
   - Different cluster structures
   - Higher/lower dimensions

2. **Hyperparameter tuning**
   - Learning rates (`lr_mu_q`, `lr_Sigma_q`)
   - KL thresholds (lower → more condensations)
   - Energy weights (`lambda_self`, `lambda_belief`, `lambda_prior`)

3. **Advanced features**
   - Enable Ouroboros Tower (`ouroboros_tower=True`)
   - Deactivate constituents (`deactivate_constituents=True`)
   - Adjust timescale filtering

4. **Analysis**
   - Compare different runs
   - Study specific condensation events
   - Analyze energy decomposition
   - Track information flow

5. **Visualization**
   - Generate complete reports with `create_analysis_report()`
   - Create custom visualizations
   - Build comparative dashboards

## Resources

- **Documentation**: See `meta/README_VISUALIZATION.md`
- **Demo script**: `examples/meta_agent_analysis_demo.py`
- **Source code**:
  - `meta/visualization.py` - Structure and dynamics
  - `meta/energy_visualization.py` - Energy analysis
  - `meta/live_monitor.py` - Real-time monitoring